In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16251355
paper_name = 'rand_grant_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table1.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 212 x 3


In [7]:
original_data.head()

,Gene,ORF,Function
0,Protein synthesis,NaN,NaN
1,RPL31A,YDL075W,60S large subunit ribosomal protein L31.e
2,RPP1B,YDL130W,60S large subunit acidic ribosomal protein L44...
3,RPL35A,YDL191W,60S large subunit ribosomal protein
4,RPL1B,YGL135W,60S large subunit ribosomal protein


In [8]:
original_data['orf'] = original_data[original_data.columns[1]].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                  Gene  ORF  Function   orf
index_input                                                                
0                                    Protein synthesis   NaN       NaN  NAN
19                        Transcription/RNA processing   NaN       NaN  NAN
44                                   Secretory pathway   NaN       NaN  NAN
45                            Vacuolar protein sorting   NaN       NaN  NAN
62                                       Glycosylation   NaN       NaN  NAN
68           Vesicular transport (Golgi network, etc.)   NaN       NaN  NAN
80                      Cell cycle and differentiation   NaN       NaN  NAN
106                                         Metabolism   NaN       NaN  NAN
107                                  Carbon metabolism   NaN       NaN  NAN
119                              Amino acid metabolism   NaN       NaN  NAN
127                                   Lipid metabolism   NaN       NaN  NAN
133         

In [12]:
original_data = original_data.loc[t,:]

In [13]:
original_data['data'] = -1

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['data']].copy()

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(195, 1)

# Load & process tested strains

In [18]:
# tested = pd.read_excel('raw_data/homozygous_diploid_obs_v1(1).0.xlsx', sheet_name='homozygous diploid_obs')

In [19]:
# tested.head()

In [20]:
# tested['orf'] = tested['ORF'].astype(str)

In [21]:
# tested['orf'] = clean_orf(tested['orf'])

In [22]:
# tested['orf'] = translate_sc(tested['orf'], to='orf')

In [23]:
# # Make sure everything translated ok
# t = looks_like_orf(tested['orf'])
# print(tested.loc[~t,])

In [24]:
# tested = tested.loc[t,:]

In [25]:
# tested_orfs = tested['orf'].unique()

In [26]:
# missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
# missing

In [27]:
# original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [28]:
data = original_data.copy()

In [29]:
dataset_ids = [1305]
datasets = datasets.reindex(index=dataset_ids)

In [30]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [31]:
data.head()

dataset_id,1305
data_type,value
orf,
YAL016W,-1
YAL035W,-1
YBL006C,-1
YBL025W,-1
YBL033C,-1


## Subset to the genes currently in SGD

In [32]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [33]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,1305
,data_type,value
gene_id,orf,
14,YAL016W,-1
33,YAL035W,-1
94,YBL006C,-1
113,YBL025W,-1
121,YBL033C,-1


# Normalize

In [34]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [35]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [36]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id       1305          
data_type       value    valuez
gene_id orf                    
14      YAL016W    -1 -4.836825
33      YAL035W    -1 -4.836825
94      YBL006C    -1 -4.836825
113     YBL025W    -1 -4.836825
121     YBL033C    -1 -4.836825

# Print out

In [37]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [45]:
# from IO.save_data_to_db3 import *

In [47]:
# save_data_to_db(data_all, paper_pmid)